In [ ]:
import pathlib
import os
import random
import copy
import re
from tqdm import tqdm, trange
from glob import glob

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import math

import scipy 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sklearn
import nibabel as nib
import imblearn
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
def get_file_path(root:str, label):
    paths = [x.replace('\\', '/') for x in glob(root, recursive=True)]
    result = []
    for i in paths:
        if 'Post' not in i:
            result.append(i)
    return [result, label]
g1_paths = get_file_path('G1/**/*.nii.gz',0)
g2_paths = get_file_path('G2/**/*.nii.gz',1)


In [ ]:
def concatenate_and_fill(file_paths, slice_range, save_path):
    image = []
    for i in trange(0, int(len(file_paths[0])/3)):
        lbl = file_paths[1]
        t1,t2,flair = [], [], []

        t1 = nib.load(file_paths[0][i*3+1]).get_fdata().astype('float32').swapaxes(0,2)
        t2 = nib.load(file_paths[0][i*3+2]).get_fdata().astype('float32').swapaxes(0,2)
        flair = nib.load(file_paths[0][i*3]).get_fdata().astype('float32').swapaxes(0,2)

        t1 = min_max_normalize(t1[slice_range[0]:slice_range[1],np.newaxis, :,:])
        t2 = min_max_normalize(t2[slice_range[0]:slice_range[1],np.newaxis, :,:])
        flair = min_max_normalize(flair[slice_range[0]:slice_range[1],np.newaxis, :,:])
        concatenated_result = np.concatenate((t1,t2,flair),axis=1)
    #     print(concatenated_result.shape, t1.shape, t2.shape, flair.shape)
    #     print(np.min(concatenated_result), np.max(concatenated_result))
        for j in concatenated_result:
            image.append(j)

    print(np.array(image).shape)
    os.makedirs(os.path.dirname(save_path+'_img.npy'), exist_ok=True)
    np.save(save_path+'_img.npy', image)


def min_max_normalize(input_array):
    for (subject_ind, subjects) in enumerate(input_array):
        for (slice_ind, slices) in enumerate(subjects):
            minimum = np.min(slices)
            maximum = np.max(slices)
            if maximum == minimum: 
                continue
            input_array[subject_ind][slice_ind] = (slices - minimum)/(maximum-minimum)
    return input_array

In [ ]:
concatenate_and_fill(g1_paths,[40,140],'concatenated_data/g1_concatenated')
concatenate_and_fill(g2_paths,[40,140],'concatenated_data/g2_concatenated')

In [ ]:
def patients_index(total_patients, ratio):
    result = []
    for i in total_patients:
        temp = [math.floor(i*x) for x in ratio]
        while sum(temp) != i:
            temp[0]+=1
        for j,v in enumerate(temp[1:]):
            temp[j+1] = temp[j] + v
        for k,v in enumerate(temp):
            if k == 0 :
                temp[k] = [0,temp[k]]
            else:
                temp[k] = [temp[k-1][1], temp[k]]
        result.append(temp)
    return result


num_subjects = [65,57,31]
ratio = [0.6, 0.2, 0.2]
patients_index(num_subjects, ratio)

In [ ]:
print(np.concatenate([[1],[2],[3]]))
a = [1,2,3]
b=[2,3]
c=[i+j for i j  in  a b]
for i, j in a, b:
    print(i,j)

In [ ]:
def patients_index(total_patients, ratio):
    result = []
    for i in total_patients:
        temp = [math.floor(i*x) for x in ratio]
        while sum(temp) != i:
            temp[0]+=1
        for j,v in enumerate(temp[1:]):
            temp[j+1] = temp[j] + v
        result.append(temp)
    return result



def construct_dataset(patients_index, file_path, save_path):
    raw_data = []
    for path in file_path:
        data = np.load(path)
        np.random.shuffle(data)
        raw_data.append(data)
    print(len(raw_data))
    num_slices_per_patient = raw_data[0][1]
    train = np.concatenate([raw_data[i][patients_index[i][0][0]:patients_index[i][0][1]] for i in range(len(raw_data))], axis=0)
    test = np.concatenate([raw_data[i][patients_index[i][1][0]:patients_index[i][1][1]] for i in range(len(raw_data))], axis=0)
    if len(patients_index) == 3:
        val = np.concatenate([data[i][patients_index[i][2][0]:patients_index[i][2][1]] for i in range(len(data))], axis=0)
    
    print(train.shape, test.shape, val.shape)












    # class0 = np.load(file_path[0])
    # np.random.shuffle(class0 )
    # class1 = np.load(file_path[1])
    # np.random.shuffle(class1)
    # class2 = np.load(file_path[2])
    # np.random.shuffle(class2)
    # num_slices_per_patient = class0.shape[1]
    # train = np.concatenate((class0[0:patients_index[0][0]], class1[0:patients_index[1][0]], class2[0:patients_index[2][0]]), axis=0)
    # val = np.concatenate((class0[patients_index[0][0]:patients_index[0][1]], class1[patients_index[1][0]:patients_index[1][1]], class2[patients_index[2][0]:patients_index[2][1]]), axis=0)
    # test = np.concatenate((class0[patients_index[0][1]:patients_index[0][2]], class1[patients_index[1][1]:patients_index[1][2]], class2[patients_index[2][1]:patients_index[2][2]]), axis=0)

    # train = np.reshape(train, (train.shape[0]*train.shape[1], train.shape[2], train.shape[3], train.shape[4]))
    # test = np.reshape(test, (test.shape[0]*test.shape[1], test.shape[2], test.shape[3], test.shape[4]))
    # val = np.reshape(val, (val.shape[0]*val.shape[1], val.shape[2], val.shape[3], val.shape[4]))
    # print(train.shape, test.shape, val.shape)

    # np.save(save_path+'training_img.npy', train)
    # np.save(save_path+'testing_img.npy', test)
    # np.save(save_path+'validation_img.npy', val)

    # np.save(save_path+'training_lbl.npy', np.concatenate((np.array([0]*((patients_index[0][0])*num_slices_per_patient)), np.array([1]*((patients_index[1][0])*num_slices_per_patient)), np.array([2]*((patients_index[2][0])*num_slices_per_patient)))))
    # np.save(save_path+'testing_lbl.npy', np.concatenate((np.array([0]*((patients_index[0][1]-patients_index[0][0])*num_slices_per_patient)), np.array([1]*((patients_index[1][1]-patients_index[1][0])*num_slices_per_patient)), np.array([2]*((patients_index[2][1]-patients_index[2][0])*num_slices_per_patient)))))
    # np.save(save_path+'validation_lbl.npy', np.concatenate((np.array([0]*((patients_index[0][2]-patients_index[0][1])*num_slices_per_patient)), np.array([1]*((patients_index[1][2]-patients_index[1][1])*num_slices_per_patient)), np.array([2]*((patients_index[2][2]-patients_index[2][1])*num_slices_per_patient)))))


In [ ]:
num_subjects = [11,24]
split_ratio = [0.6, 0.2]
file_paths = ['concatenated_data/g1_concatenated_img.npy', 'concatenated_data/g2_concatenated_img.npy']
construct_dataset(patients_index(num_subjects, split_ratio), file_paths, [])

In [ ]:
t1 = nib.load('G1/RR-044/FLAIR_MNI_reg.nii.gz').get_fdata().astype('float32').swapaxes(0,2)
concat = np.load('concatenated_data/g1_concatenated_img.npy')
print(t1.shape)

plt.imshow(concat[20][1][:][:], cmap="grey")
